In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
from scipy.optimize import minimize,minimize_scalar, curve_fit
import seaborn as sns
sns.set()
sns.set_palette("husl")

# Load User-Defined Functions:

# Load in Matrix:

In [2]:
referenceDF = pd.read_csv('Data_To_Share/sequence_level_data_Jurkat.csv')
referenceDF_logratio = pd.read_csv('Data_To_Share/sequence_level_data_Jurkat_T0T4_logRatios.csv')

In [3]:
#Some of the CXCL7 padding sequences have AREs -- we can account for these if desired...

cxcl7_list = ['SUPV3L1|10:70968792-70968830','TRPT1|11:63991271-63991346','ART4|12:14982266-14982303','POLE2|14:50116899-50116969','NMRAL1|16:4511716-4511779','ADPRM|17:10614462-10614520','NUP85|17:73231775-73231829','PPP1R15A|19:49379231-49379294','PQLC3|2:11300834-11300874','FASTKD1|2:170386287-170386333','TFPI|2:188343337-188343401','YBEY|21:47717549-47717616','ALG1L|3:125648118-125648193','HELQ|4:84328529-84328604','TMEM171|5:72427558-72427617','IL4|5:132018280-132018347','PCDHA11|5:140251122-140251185','PCDHA12|5:140257437-140257474','GIN1|5:102423545-102423600','HLA-DQA1|6:32610542-32610561','CCDC132|7:92905660-92905721','NAPRT|8:144656955-144657006']
CXCL7_ARE_list = []
for region in referenceDF.region:
    if region in cxcl7_list:
        CXCL7_ARE_list.append(1)
    else:
        CXCL7_ARE_list.append(0)
referenceDF['CXCL7_ARE']=CXCL7_ARE_list

In [5]:
len(cxcl7_list),sum(CXCL7_ARE_list)

(22, 60)

# Predictive Power of Different Categorizations

We start with a bunch of oligo ratios and different categories.  Do LOCO and for each category predict a mean.  See how well predictions correlate with left out oligos.  Iterate over all chromosomes.

## ARED Plus Clusters

In [4]:
chrom_number = []
for region in referenceDF.region:
    chrom_number.append(region.split('|')[1].split(':')[0])
referenceDF['chrom_number']=chrom_number

In [5]:
# Accounts for CXCL7:

for are_category in ['parent_motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.nanmedian(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])
        
for are_category in ['motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.iscontrol==1)&(referenceDF.CXCL7_ARE==0)]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.iscontrol==1)&(referenceDF.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.nanmedian(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

parent_motif_AREs_numbered_BakheetPlus effect_size_T0_GC_resid 0.12988807700915642
parent_motif_AREs_numbered_BakheetPlus effect_size_T4T0_GC_resid 0.30601023785419845
motif_AREs_numbered_BakheetPlus ratios_T0_GC_resid 0.08862665534333984
motif_AREs_numbered_BakheetPlus ratios_T4T0_GC_resid 0.2548912841693636


In [1]:
# Note: some of the correlations above are slightly different from published values due to exclusion of slightly different set of sequences

## Effective Length

In [6]:
# First make "effective perfect length" variable from length and registration:
referenceDF['effective_perfect_length'] = referenceDF.ARE_registration_perfect + referenceDF.ARE_length_perfect


# It helps to make a "parent effective perfect length" variable:
referenceDF_byOligo = referenceDF.copy()
referenceDF_byOligo.index = referenceDF_byOligo.ids
parent_effective_perfect_length_list = []
for parent_oligo in referenceDF_byOligo.parent_control_oligo:
    if parent_oligo in referenceDF_byOligo.ids.values:
        parent_effective_perfect_length_list.append(referenceDF_byOligo.loc[parent_oligo,'effective_perfect_length'])
    else:
        parent_effective_perfect_length_list.append(np.nan)
referenceDF['parent_effective_perfect_length'] = parent_effective_perfect_length_list

In [7]:
#Account for CXCL7:

for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
#     are_category = 'parent_ARE'
#     figure_of_merit = 'effect_size_T0_GC_resid'
    test_predictions = []
    test_actual = []
    for chrom in referenceDF.chrom_number.unique():
        train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)] #it had iscontrol==1 above
        test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)]

        train_df.index = train_df.ids
        test_df.index = test_df.ids
        
        predictions_dict = {}
        
        for parent_effective_perfect_length in range(5,22):
            predictions_dict[parent_effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.parent_effective_perfect_length==parent_effective_perfect_length])
        
        for parent_effective_perfect_length,test_fom in zip(test_df.parent_effective_perfect_length,test_df[figure_of_merit]):
            if parent_effective_perfect_length<22:
                if not pd.isnull(test_fom):
                    test_predictions.append(predictions_dict[parent_effective_perfect_length])
                    test_actual.append(test_fom)

    print(figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])



for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
#     are_category = 'parent_ARE'
#     figure_of_merit = 'effect_size_T0_GC_resid'
    test_predictions = []
    test_actual = []
    for chrom in referenceDF.chrom_number.unique():
        train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.CXCL7_ARE==0)] #it had iscontrol==1 above
        test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.CXCL7_ARE==0)]

        train_df.index = train_df.ids
        test_df.index = test_df.ids
        
        predictions_dict = {}
        
        for effective_perfect_length in range(5,22):
            mask = train_df.effective_perfect_length==effective_perfect_length
            mask = mask.fillna(False)
            predictions_dict[effective_perfect_length]=np.mean(train_df[figure_of_merit][mask])
        
        for effective_perfect_length,test_fom in zip(test_df.effective_perfect_length,test_df[figure_of_merit]):
            if not pd.isnull(effective_perfect_length):
                if effective_perfect_length<22:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[effective_perfect_length])
                        test_actual.append(test_fom)

    print(figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

effect_size_T0_GC_resid 0.15839698465117696
effect_size_T4T0_GC_resid 0.3696980185478312
ratios_T0_GC_resid 0.14078280483265623
ratios_T4T0_GC_resid 0.3095853851658214


## Naive Effective Length Pentamers

In [8]:
#Groups: 5, 6-9, 10-13, 14-17, 18-21 (effective length).  Add one for imperfect?
david_groups_perfect = []
for effective_perfect_length in referenceDF.effective_perfect_length:
    if not pd.isnull(effective_perfect_length):
        if effective_perfect_length<=5:
            david_groups_perfect.append(1)
        elif effective_perfect_length<=9:
            david_groups_perfect.append(2)
        elif effective_perfect_length<=13:
            david_groups_perfect.append(3)
        elif effective_perfect_length<=17:
            david_groups_perfect.append(4)
        elif effective_perfect_length<=22:
            david_groups_perfect.append(5)
        elif effective_perfect_length>22:
            david_groups_perfect.append(6)
        else:
            david_groups_perfect.append(np.nan)
    else:
        david_groups_perfect.append(np.nan)
referenceDF['david_groups_perfect']=david_groups_perfect

parent_david_groups_perfect = []
referenceDF_byOligo = referenceDF.copy()
referenceDF_byOligo.index = referenceDF.ids
for oligo,parent_oligo in zip(referenceDF.ids.values,referenceDF.parent_control_oligo.values):
    if not pd.isnull(parent_oligo):
        parent_david_groups_perfect.append(referenceDF_byOligo.loc[parent_oligo,'david_groups_perfect'])
    else:
        parent_david_groups_perfect.append(np.nan)
        
referenceDF['parent_david_groups_perfect']=parent_david_groups_perfect


In [9]:
#Account for CXCL7

for are_category in ['parent_david_groups_perfect']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')&(referenceDF.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category > 1)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category>1)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])
        
for are_category in ['david_groups_perfect']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.iscontrol==1)&(referenceDF.CXCL7_ARE==0)]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.iscontrol==1)&(referenceDF.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category > 1)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category > 1)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

parent_david_groups_perfect effect_size_T0_GC_resid 0.14972015078440315
parent_david_groups_perfect effect_size_T4T0_GC_resid 0.3529200985582243
david_groups_perfect ratios_T0_GC_resid 0.1149938185724824
david_groups_perfect ratios_T4T0_GC_resid 0.29949281424547497


## Mann Whitney U Test For Significance of Improvement

In [19]:
# It helps to make a "parent_ARE_length_perfect" and a "parent_ARE_registration_perfect" variable:
referenceDF_byOligo = referenceDF.copy()
referenceDF_byOligo.index = referenceDF_byOligo.ids
parent_ARE_length_perfect_list = []
parent_ARE_registration_perfect_list = []
for parent_oligo in referenceDF_byOligo.parent_control_oligo:
    if parent_oligo in referenceDF_byOligo.ids.values:
        parent_ARE_length_perfect_list.append(referenceDF_byOligo.loc[parent_oligo,'ARE_length_perfect'])
        parent_ARE_registration_perfect_list.append(referenceDF_byOligo.loc[parent_oligo,'ARE_registration_perfect'])
    else:
        parent_ARE_length_perfect_list.append(np.nan)
        parent_ARE_registration_perfect_list.append(np.nan)
referenceDF['parent_ARE_length_perfect'] = parent_ARE_length_perfect_list
referenceDF['parent_ARE_registration_perfect'] = parent_ARE_registration_perfect_list

In [20]:
#Do it out of sample so I don't have to worry about degrees of freedom for each test...


#Minimize the squared deviations from the mean.
def squared_deviation(f1x,f1y,f2x,f2y,f3x,f3y,f4x,f4y):
    running_deviation = 0
    for i in range(1,25):
        f1x = np.array(f1x)
        f1y = np.array(f1y)
        f2x = np.array(f2x)
        f2y = np.array(f2y)
        f3x = np.array(f3x)
        f3y = np.array(f3y)
        f4x = np.array(f4x)
        f4y = np.array(f4y)        

        y_vals = f1y[f1x==i]
        y_vals = np.append(y_vals,f2y[f2x==i])
        y_vals = np.append(y_vals,f3y[f3x==i])
        y_vals = np.append(y_vals,f4y[f4x==i])
        if len(y_vals)>1:
            running_deviation += np.var(y_vals)*len(y_vals)
    return running_deviation

rangemin = 5
rangemax = 22

for are_category in ['parent_motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')]
            train_df.index = train_df.ids
            test_df.index = test_df.ids
            
            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
                        
        Bakheet_resids = np.array(test_predictions)-np.array(test_actual)

            

        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')] #it had iscontrol==1 above
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.top_ARE_deleted==1)&(referenceDF.hap=='mutated_ARE')]

            train_df.index = train_df.ids
            test_df.index = test_df.ids

            def offset_dev(offset):
                referenceDF_inUse = train_df.copy()
                registration=0
                f0x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f0y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=1
                f1x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f1y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=2
                f2x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f2y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=3
                f3x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f3y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                return squared_deviation(f0x,f0y,f1x,f1y,f2x,f2y,f3x,f3y)

            aaa = minimize(offset_dev,.001)
            shift = aaa.x[0]

            predictions_dict = {}

            for parent_effective_perfect_length in range(rangemin,rangemax):
                predictions_dict[parent_effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.parent_effective_perfect_length==parent_effective_perfect_length]+shift*train_df.parent_ARE_registration_perfect[train_df.parent_effective_perfect_length==parent_effective_perfect_length])

            for parent_effective_perfect_length,test_fom,parent_ARE_registration_perfect in zip(test_df.parent_effective_perfect_length,test_df[figure_of_merit],test_df.parent_ARE_registration_perfect):
                if parent_effective_perfect_length<rangemax:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[parent_effective_perfect_length]-shift*parent_ARE_registration_perfect)
                        test_actual.append(test_fom)



        David_resids = np.array(test_predictions)-np.array(test_actual)
                        
                        
        statistic,pvalue = st.mannwhitneyu(Bakheet_resids**2,David_resids**2)
        print(are_category, figure_of_merit, pvalue)
        
for are_category in ['motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)&(referenceDF.iscontrol==1)]
            test_df = referenceDF[(referenceDF.chrom_number==chrom)&(referenceDF.iscontrol==1)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
                        
        Bakheet_resids = np.array(test_predictions)-np.array(test_actual)
            
        test_predictions = []
        test_actual = []
        for chrom in referenceDF.chrom_number.unique():
            train_df = referenceDF[(referenceDF.chrom_number!=chrom)] #it had iscontrol==1 above
            test_df = referenceDF[(referenceDF.chrom_number==chrom)]

            train_df.index = train_df.ids
            test_df.index = test_df.ids

            def offset_dev(offset):
                referenceDF_inUse = train_df.copy()
                registration=0
                f0x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f0y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=1
                f1x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f1y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=2
                f2x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f2y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=3
                f3x = referenceDF_inUse.parent_ARE_length_perfect[referenceDF_inUse.parent_ARE_registration_perfect==registration]+registration
                f3y = referenceDF_inUse[figure_of_merit][referenceDF_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                return squared_deviation(f0x,f0y,f1x,f1y,f2x,f2y,f3x,f3y)

            aaa = minimize(offset_dev,.001)
            shift = aaa.x[0]


            predictions_dict = {}

            for effective_perfect_length in range(rangemin,rangemax):
                predictions_dict[effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.effective_perfect_length==effective_perfect_length]-shift*train_df.ARE_registration_perfect[train_df.effective_perfect_length==effective_perfect_length])

            for effective_perfect_length,test_fom,ARE_registration_perfect in zip(test_df.effective_perfect_length,test_df[figure_of_merit],test_df.ARE_registration_perfect):
                if effective_perfect_length<rangemax:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[effective_perfect_length]+shift*ARE_registration_perfect)
                        test_actual.append(test_fom)


                        
        David_resids = np.array(test_predictions)-np.array(test_actual)
        statistic,pvalue = st.mannwhitneyu(Bakheet_resids**2,David_resids**2)
        print(are_category, figure_of_merit, pvalue)


        
        

parent_motif_AREs_numbered_BakheetPlus effect_size_T0_GC_resid 0.3371002781988618
parent_motif_AREs_numbered_BakheetPlus effect_size_T4T0_GC_resid 0.00421447853462852
motif_AREs_numbered_BakheetPlus ratios_T0_GC_resid 0.06504543848154937
motif_AREs_numbered_BakheetPlus ratios_T4T0_GC_resid 3.851469606761032e-12


# Now Do This All Again For Log-Ratios:

In [21]:
#Some of the CXCL7 padding sequences have AREs -- we can account for these if desired...

cxcl7_list = ['SUPV3L1|10:70968792-70968830','TRPT1|11:63991271-63991346','ART4|12:14982266-14982303','POLE2|14:50116899-50116969','NMRAL1|16:4511716-4511779','ADPRM|17:10614462-10614520','NUP85|17:73231775-73231829','PPP1R15A|19:49379231-49379294','PQLC3|2:11300834-11300874','FASTKD1|2:170386287-170386333','TFPI|2:188343337-188343401','YBEY|21:47717549-47717616','ALG1L|3:125648118-125648193','HELQ|4:84328529-84328604','TMEM171|5:72427558-72427617','IL4|5:132018280-132018347','PCDHA11|5:140251122-140251185','PCDHA12|5:140257437-140257474','GIN1|5:102423545-102423600','HLA-DQA1|6:32610542-32610561','CCDC132|7:92905660-92905721','NAPRT|8:144656955-144657006']
CXCL7_ARE_list = []
for region in referenceDF_logratio.region:
    if region in cxcl7_list:
        CXCL7_ARE_list.append(1)
    else:
        CXCL7_ARE_list.append(0)
referenceDF_logratio['CXCL7_ARE']=CXCL7_ARE_list

In [24]:
chrom_number = []
for region in referenceDF_logratio.region:
    if not pd.isnull(region):
        chrom_number.append(region.split('|')[1].split(':')[0])
    else:
        chrom_number.append(np.nan)
referenceDF_logratio['chrom_number']=chrom_number

## ARED-Plus Clusters

In [25]:
# Accounts for CXCL7:

for are_category in ['parent_motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.nanmedian(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])
        
for are_category in ['motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.iscontrol==1)&(referenceDF_logratio.CXCL7_ARE==0)]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.iscontrol==1)&(referenceDF_logratio.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.nanmedian(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

parent_motif_AREs_numbered_BakheetPlus effect_size_T0_GC_resid 0.12219426353873165
parent_motif_AREs_numbered_BakheetPlus effect_size_T4T0_GC_resid 0.30563617833484474
motif_AREs_numbered_BakheetPlus ratios_T0_GC_resid 0.07155643391110716
motif_AREs_numbered_BakheetPlus ratios_T4T0_GC_resid 0.24702946365276734


## Effective Length

In [26]:
# First make "effective perfect length" variable from length and registration:
referenceDF_logratio['effective_perfect_length'] = referenceDF_logratio.ARE_registration_perfect + referenceDF_logratio.ARE_length_perfect


# It helps to make a "parent effective perfect length" variable:
referenceDF_logratio_byOligo = referenceDF_logratio.copy()
referenceDF_logratio_byOligo.index = referenceDF_logratio_byOligo.ids
parent_effective_perfect_length_list = []
for parent_oligo in referenceDF_logratio_byOligo.parent_control_oligo:
    if parent_oligo in referenceDF_logratio_byOligo.ids.values:
        parent_effective_perfect_length_list.append(referenceDF_logratio_byOligo.loc[parent_oligo,'effective_perfect_length'])
    else:
        parent_effective_perfect_length_list.append(np.nan)
referenceDF_logratio['parent_effective_perfect_length'] = parent_effective_perfect_length_list

In [27]:
#Account for CXCL7:

for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
#     are_category = 'parent_ARE'
#     figure_of_merit = 'effect_size_T0_GC_resid'
    test_predictions = []
    test_actual = []
    for chrom in referenceDF_logratio.chrom_number.unique():
        train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)] #it had iscontrol==1 above
        test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)]

        train_df.index = train_df.ids
        test_df.index = test_df.ids
        
        predictions_dict = {}
        
        for parent_effective_perfect_length in range(5,22):
            predictions_dict[parent_effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.parent_effective_perfect_length==parent_effective_perfect_length])
        
        for parent_effective_perfect_length,test_fom in zip(test_df.parent_effective_perfect_length,test_df[figure_of_merit]):
            if parent_effective_perfect_length<22:
                if not pd.isnull(test_fom):
                    test_predictions.append(predictions_dict[parent_effective_perfect_length])
                    test_actual.append(test_fom)

    print(figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])



for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
#     are_category = 'parent_ARE'
#     figure_of_merit = 'effect_size_T0_GC_resid'
    test_predictions = []
    test_actual = []
    for chrom in referenceDF_logratio.chrom_number.unique():
        train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.CXCL7_ARE==0)] #it had iscontrol==1 above
        test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.CXCL7_ARE==0)]

        train_df.index = train_df.ids
        test_df.index = test_df.ids
        
        predictions_dict = {}
        
        for effective_perfect_length in range(5,22):
            mask = train_df.effective_perfect_length==effective_perfect_length
            mask = mask.fillna(False)
            predictions_dict[effective_perfect_length]=np.mean(train_df[figure_of_merit][mask])
        
        for effective_perfect_length,test_fom in zip(test_df.effective_perfect_length,test_df[figure_of_merit]):
            if not pd.isnull(effective_perfect_length):
                if effective_perfect_length<22:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[effective_perfect_length])
                        test_actual.append(test_fom)

    print(figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

effect_size_T0_GC_resid 0.1472346682873706
effect_size_T4T0_GC_resid 0.3867410118526146
ratios_T0_GC_resid 0.13692621237606908
ratios_T4T0_GC_resid 0.32030577499138385


## Naive Effective Length Pentamers

In [28]:
#Groups: 5, 6-9, 10-13, 14-17, 18-21 (effective length).  Add one for imperfect?
david_groups_perfect = []
for effective_perfect_length in referenceDF_logratio.effective_perfect_length:
    if not pd.isnull(effective_perfect_length):
        if effective_perfect_length<=5:
            david_groups_perfect.append(1)
        elif effective_perfect_length<=9:
            david_groups_perfect.append(2)
        elif effective_perfect_length<=13:
            david_groups_perfect.append(3)
        elif effective_perfect_length<=17:
            david_groups_perfect.append(4)
        elif effective_perfect_length<=22:
            david_groups_perfect.append(5)
        elif effective_perfect_length>22:
            david_groups_perfect.append(6)
        else:
            david_groups_perfect.append(np.nan)
    else:
        david_groups_perfect.append(np.nan)
referenceDF_logratio['david_groups_perfect']=david_groups_perfect

parent_david_groups_perfect = []
referenceDF_logratio_byOligo = referenceDF_logratio.copy()
referenceDF_logratio_byOligo.index = referenceDF_logratio.ids
for oligo,parent_oligo in zip(referenceDF_logratio.ids.values,referenceDF_logratio.parent_control_oligo.values):
    if not pd.isnull(parent_oligo):
        parent_david_groups_perfect.append(referenceDF_logratio_byOligo.loc[parent_oligo,'david_groups_perfect'])
    else:
        parent_david_groups_perfect.append(np.nan)
        
referenceDF_logratio['parent_david_groups_perfect']=parent_david_groups_perfect



In [30]:
#Account for CXCL7

for are_category in ['parent_david_groups_perfect']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')&(referenceDF_logratio.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category > 1)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category>1)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])
        
for are_category in ['david_groups_perfect']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.iscontrol==1)&(referenceDF_logratio.CXCL7_ARE==0)]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.iscontrol==1)&(referenceDF_logratio.CXCL7_ARE==0)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category > 1)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category > 1)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
        print(are_category, figure_of_merit, np.corrcoef(test_predictions,test_actual)[0,1])

parent_david_groups_perfect effect_size_T0_GC_resid 0.14471975732036033
parent_david_groups_perfect effect_size_T4T0_GC_resid 0.3621910843279327
david_groups_perfect ratios_T0_GC_resid 0.10716883090969442
david_groups_perfect ratios_T4T0_GC_resid 0.30679478908547325


## Mann Whitney U Test For Significance of Improvement

In [31]:
# It helps to make a "parent_ARE_length_perfect" and a "parent_ARE_registration_perfect" variable:
referenceDF_logratio_byOligo = referenceDF_logratio.copy()
referenceDF_logratio_byOligo.index = referenceDF_logratio_byOligo.ids
parent_ARE_length_perfect_list = []
parent_ARE_registration_perfect_list = []
for parent_oligo in referenceDF_logratio_byOligo.parent_control_oligo:
    if parent_oligo in referenceDF_logratio_byOligo.ids.values:
        parent_ARE_length_perfect_list.append(referenceDF_logratio_byOligo.loc[parent_oligo,'ARE_length_perfect'])
        parent_ARE_registration_perfect_list.append(referenceDF_logratio_byOligo.loc[parent_oligo,'ARE_registration_perfect'])
    else:
        parent_ARE_length_perfect_list.append(np.nan)
        parent_ARE_registration_perfect_list.append(np.nan)
referenceDF_logratio['parent_ARE_length_perfect'] = parent_ARE_length_perfect_list
referenceDF_logratio['parent_ARE_registration_perfect'] = parent_ARE_registration_perfect_list

In [32]:
#Do it out of sample so I don't have to worry about degrees of freedom for each test...


#Minimize the squared deviations from the mean.
def squared_deviation(f1x,f1y,f2x,f2y,f3x,f3y,f4x,f4y):
    running_deviation = 0
    for i in range(1,25):
        f1x = np.array(f1x)
        f1y = np.array(f1y)
        f2x = np.array(f2x)
        f2y = np.array(f2y)
        f3x = np.array(f3x)
        f3y = np.array(f3y)
        f4x = np.array(f4x)
        f4y = np.array(f4y)        

        y_vals = f1y[f1x==i]
        y_vals = np.append(y_vals,f2y[f2x==i])
        y_vals = np.append(y_vals,f3y[f3x==i])
        y_vals = np.append(y_vals,f4y[f4x==i])
        if len(y_vals)>1:
            running_deviation += np.var(y_vals)*len(y_vals)
    return running_deviation

rangemin = 5
rangemax = 22

for are_category in ['parent_motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['effect_size_T0_GC_resid','effect_size_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')]
            train_df.index = train_df.ids
            test_df.index = test_df.ids
            
            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
                        
        Bakheet_resids = np.array(test_predictions)-np.array(test_actual)

            

        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')] #it had iscontrol==1 above
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.top_ARE_deleted==1)&(referenceDF_logratio.hap=='mutated_ARE')]

            train_df.index = train_df.ids
            test_df.index = test_df.ids

            def offset_dev(offset):
                referenceDF_logratio_inUse = train_df.copy()
                registration=0
                f0x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f0y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=1
                f1x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f1y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=2
                f2x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f2y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=3
                f3x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f3y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                return squared_deviation(f0x,f0y,f1x,f1y,f2x,f2y,f3x,f3y)

            aaa = minimize(offset_dev,.001)
            shift = aaa.x[0]

            predictions_dict = {}

            for parent_effective_perfect_length in range(rangemin,rangemax):
                predictions_dict[parent_effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.parent_effective_perfect_length==parent_effective_perfect_length]+shift*train_df.parent_ARE_registration_perfect[train_df.parent_effective_perfect_length==parent_effective_perfect_length])

            for parent_effective_perfect_length,test_fom,parent_ARE_registration_perfect in zip(test_df.parent_effective_perfect_length,test_df[figure_of_merit],test_df.parent_ARE_registration_perfect):
                if parent_effective_perfect_length<rangemax:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[parent_effective_perfect_length]-shift*parent_ARE_registration_perfect)
                        test_actual.append(test_fom)



        David_resids = np.array(test_predictions)-np.array(test_actual)
                        
                        
        statistic,pvalue = st.mannwhitneyu(Bakheet_resids**2,David_resids**2)
        print(are_category, figure_of_merit, pvalue)
        
for are_category in ['motif_AREs_numbered_BakheetPlus']:
    for figure_of_merit in ['ratios_T0_GC_resid','ratios_T4T0_GC_resid']:
    #     are_category = 'parent_ARE'
    #     figure_of_merit = 'effect_size_T0_GC_resid'
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)&(referenceDF_logratio.iscontrol==1)]
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)&(referenceDF_logratio.iscontrol==1)]

            predictions_dict = {}
            for category in train_df[are_category].unique():
                if (category != 0)&(category<7):
                    predictions_dict[category]=np.mean(train_df[figure_of_merit][train_df[are_category]==category])

            for test_category,test_fom in zip(test_df[are_category],test_df[figure_of_merit]):
                if (test_category!=0)&(test_category<7):
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[test_category])
                        test_actual.append(test_fom)
                        
        Bakheet_resids = np.array(test_predictions)-np.array(test_actual)
            
        test_predictions = []
        test_actual = []
        for chrom in referenceDF_logratio.chrom_number.unique():
            train_df = referenceDF_logratio[(referenceDF_logratio.chrom_number!=chrom)] #it had iscontrol==1 above
            test_df = referenceDF_logratio[(referenceDF_logratio.chrom_number==chrom)]

            train_df.index = train_df.ids
            test_df.index = test_df.ids

            def offset_dev(offset):
                referenceDF_logratio_inUse = train_df.copy()
                registration=0
                f0x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f0y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=1
                f1x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f1y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=2
                f2x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f2y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                registration=3
                f3x = referenceDF_logratio_inUse.parent_ARE_length_perfect[referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+registration
                f3y = referenceDF_logratio_inUse[figure_of_merit][referenceDF_logratio_inUse.parent_ARE_registration_perfect==registration]+offset*registration
                return squared_deviation(f0x,f0y,f1x,f1y,f2x,f2y,f3x,f3y)

            aaa = minimize(offset_dev,.001)
            shift = aaa.x[0]


            predictions_dict = {}

            for effective_perfect_length in range(rangemin,rangemax):
                predictions_dict[effective_perfect_length]=np.mean(train_df[figure_of_merit][train_df.effective_perfect_length==effective_perfect_length]-shift*train_df.ARE_registration_perfect[train_df.effective_perfect_length==effective_perfect_length])

            for effective_perfect_length,test_fom,ARE_registration_perfect in zip(test_df.effective_perfect_length,test_df[figure_of_merit],test_df.ARE_registration_perfect):
                if effective_perfect_length<rangemax:
                    if not pd.isnull(test_fom):
                        test_predictions.append(predictions_dict[effective_perfect_length]+shift*ARE_registration_perfect)
                        test_actual.append(test_fom)


                        
        David_resids = np.array(test_predictions)-np.array(test_actual)
        statistic,pvalue = st.mannwhitneyu(Bakheet_resids**2,David_resids**2)
        print(are_category, figure_of_merit, pvalue)


        
        

parent_motif_AREs_numbered_BakheetPlus effect_size_T0_GC_resid 0.33277665459891315
parent_motif_AREs_numbered_BakheetPlus effect_size_T4T0_GC_resid 0.0013465789645256332
motif_AREs_numbered_BakheetPlus ratios_T0_GC_resid 0.006796850275162462
motif_AREs_numbered_BakheetPlus ratios_T4T0_GC_resid 0.005655734532549821
